### **Enhanced Portfolio Optimization Workflow**
This document demonstrates the core backend logic of the portfolio optimization application. It covers data fetching, GARCH volatility forecasting, covariance construction, and running the three optimization models.

#### 1. Configuration & Data Fetching

In [1]:
import pandas as pd
import numpy as np
import sys
import os

# Add the project root to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.data.fetcher import fetch_price_data
from src.models.garch_model import get_garch_volatility_forecast
from src.optimization.mvo import calculate_mvo_weights
from src.optimization.risk_parity import calculate_risk_parity_weights
from src.optimization.robust_mvo import calculate_enhanced_mvo_weights
from src.models.utils import calculate_returns, calculate_covariance_matrix
from src.optimization.performance import calculate_portfolio_performance, calculate_risk_contribution

# Set display options for pandas
pd.set_option('display.float_format', lambda x: '%.4f' % x)

In [2]:
tickers = ['SPY', 'TLT', 'GLD', 'DBC'] # A classic multi-asset portfolio
start_date = '2018-01-01'
end_date = '2023-12-31'
risk_free_rate = 0.02

price_data = fetch_price_data(tickers, start_date, end_date)
log_returns = calculate_returns(price_data)

print("Fetched Price Data (tail):")
display(price_data.tail())

Fetched Price Data (tail):


/home/timothykimutai/projects/volatility-prediction/volatility-prediction/src/data/fetcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.dropna(axis=1, how="all", inplace=True)
/home/timothykimutai/projects/volatility-prediction/volatility-prediction/src/data/fetcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  price_data.ffill(inplace=True)
/home/timothykimutai/projects/volatility-prediction/volatility-prediction/src/data/fetcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pa

Ticker,DBC,GLD,SPY,TLT
Date,,,,
2023-12-22,21.1034,190.2700,464.8744,91.8894
2023-12-26,21.3122,191.7200,466.8374,92.1507
2023-12-27,21.2173,192.5900,467.6814,93.7835
2023-12-28,21.0369,191.4700,467.8581,93.1024
2023-12-29,20.9230,191.1700,466.5037,92.2626


#### 2. GARCH Volatility Forecasting

In [3]:
garch_vol, garch_success_map = get_garch_volatility_forecast(log_returns, horizon=63)

print("Annualized Volatility Forecasts from GARCH (or fallback):")
display(garch_vol)

Annualized Volatility Forecasts from GARCH (or fallback):


DBC   0.1804
GLD   0.1386
SPY   0.1688
TLT   0.1731
dtype: float64

#### 3. Input Preparation (Expected Returns & Covariance)

In [4]:
annual_mean_returns = log_returns.mean() * 252
cov_matrix = calculate_covariance_matrix(
    log_returns, 
    use_garch_vol=True, 
    garch_vol_forecasts=garch_vol,
    shrinkage=True # Use Ledoit-Wolf on correlation
)

print("Annualized Expected Returns:")
display(annual_mean_returns)

print("\nForecasted Covariance Matrix:")
display(cov_matrix)

Annualized Expected Returns:


Ticker
DBC    0.0603
GLD    0.0708
SPY    0.1121
TLT   -0.0168
dtype: float64


Forecasted Covariance Matrix:


Ticker,DBC,GLD,SPY,TLT
Ticker,,,,
DBC,0.0325,0.0062,0.0109,-0.0047
GLD,0.0062,0.0192,0.0021,0.0067
SPY,0.0109,0.0021,0.0285,-0.0055
TLT,-0.0047,0.0067,-0.0055,0.0300


#### 4. Running Optimizers

In [5]:
# MVO (Max Sharpe)
mvo_weights, mvo_perf = calculate_mvo_weights(annual_mean_returns, cov_matrix, risk_free_rate)

# Risk-Parity
rp_weights = calculate_risk_parity_weights(cov_matrix)
rp_perf = calculate_portfolio_performance(rp_weights, annual_mean_returns, cov_matrix, risk_free_rate)

# Enhanced MVO
robust_mu_uncertainty = (log_returns.std() / np.sqrt(len(log_returns))) * np.sqrt(252) * 0.5
enhanced_weights, enhanced_perf = calculate_enhanced_mvo_weights(
    mu=annual_mean_returns,
    cov_matrix=cov_matrix,
    risk_free_rate=risk_free_rate,
    w_ref=rp_weights,
    epsilon=robust_mu_uncertainty,
    rho_rp=1.0,
    tau_l2=1e-5
)

results_df = pd.DataFrame({
    'MVO': mvo_weights,
    'Risk-Parity': rp_weights,
    'Enhanced MVO': enhanced_weights
})

print("Optimal Portfolio Weights:")
display(results_df * 100)

Optimal Portfolio Weights:


,MVO,Risk-Parity,Enhanced MVO
Ticker,,,
DBC,0.0000,21.4975,22.0426
GLD,43.0158,25.2420,30.9974
SPY,56.9842,25.2900,46.9601
TLT,0.0000,27.9705,0.0000


#### 5. Performance and Risk Analysis

In [6]:
perf_data = {
    'MVO': mvo_perf,
    'Risk-Parity': rp_perf,
    'Enhanced MVO': enhanced_perf
}
perf_df = pd.DataFrame(perf_data).T

print("Portfolio Performance Metrics:")
display(perf_df)


risk_contribs_df = pd.DataFrame()
for name, w in results_df.items():
    risk_contribs_df[name] = calculate_risk_contribution(w, cov_matrix)
    
print("\nRisk Contributions (%):")
display(risk_contribs_df * 100)

Portfolio Performance Metrics:


,expected_return,volatility,sharpe_ratio
MVO,0.0943,0.1176,0.6322
Risk-Parity,0.0545,0.0928,0.3713
Enhanced MVO,0.0879,0.1158,0.5860



Risk Contributions (%):


,MVO,Risk-Parity,Enhanced MVO
Ticker,,,
DBC,0.0000,24.9190,23.3493
GLD,29.3963,25.1425,19.1503
SPY,70.6037,25.0367,57.5004
TLT,-0.0000,24.9018,-0.0000
